<a href="https://colab.research.google.com/github/shobhit10058/MachineLearning/blob/master/Assignment_3_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install torch

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
def read_data(file):
  data = []
  f = open(file)
  while(True):
    s = f.readline()
    if len(s) == 0:
      break
    curr = list(s.split(','))
    data.append((torch.tensor(list(map(float, curr[:-1]))), int(curr[-1])))
  return data

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

batch_size = 5

train_dataset = read_data('drive/MyDrive/data_ass_3/optdigits.tra')
test_dataset = read_data('drive/MyDrive/data_ass_3/optdigits.tes')

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

for i, (images, labels) in enumerate(train_loader): 
    print(i, images, labels, sep="\n")
    if i == 5:
      break

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch import optim

model = nn.Sequential(nn.Linear(64, 3),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

num_epochs = 5
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    running_loss = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images
        labels = labels
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {acc} %')